# Demo While Activity with ShiftAmount Activity
The combination of a while activity with a shift amount activity can be used to represent the loading or unloading of a vessel, where coordination of the process is represented on the granularity of the amount shifted in one iteration.

In [1]:
import datetime, time
import simpy

import shapely.geometry

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of Sites

In [2]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

data_from_site = {
    "env": my_env,
    "name": "Winlocatie",
    "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5b",
    "geometry": location_from_site,
    "capacity": 10,
    "level": 10,
}

location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)

data_to_site = {
    "env": my_env,
    "name": "Dumplocatie",
    "ID": "6dbbbdf5-4589-11e9-82b2-b469212bff5b",
    "geometry": location_to_site,
    "capacity": 10,
    "level": 0,
}

from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

## Creation of Vessel

In [3]:
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

def compute_v_provider(v_empty, v_full):
    return lambda x: 10

# TSHD variables
data_hopper = {
    "env": my_env,
    "name": "Hopper 01",
    "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",
    "geometry": location_from_site,
    "loading_rate": 1,
    "unloading_rate": 1,
    "capacity": 5,
    "compute_v": compute_v_provider(5, 4.5),
}

hopper = TransportProcessingResource(**data_hopper)

## Definition of Shift Amount activity

In [4]:
shift_amount_activity_loading_data = {
    "env": my_env,
    "name": "Transfer MP",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff52",
    "registry": registry,
    "processor": hopper,
    "origin": from_site,
    "destination": hopper,
    "amount": 1,
    "duration": 20,
    "postpone_start": True,
}
activity = model.ShiftAmountActivity(**shift_amount_activity_loading_data)

while_data = {
    "env": my_env,
    "name": "while",
    "registry": registry,
    "sub_processes": [activity],
    "condition_event": [{"type":"container", "concept": hopper, "state":"full"}],
    "postpone_start": False,
}
while_activity = model.WhileActivity(**while_data)


## Run simulation

In [5]:
my_env.run()

In [6]:
display(plot.get_log_dataframe(while_activity, [activity, while_activity]))

,Activity,Timestamp,ActivityState,type,ref
0,while,1970-01-01 00:00:00,START,NaN,NaN
1,while,1970-01-01 00:00:00,START,subprocess,5e5ddf8e-d6f0-4a8d-9ed9-ea2c61e1beda
2,while,1970-01-01 00:00:20,STOP,subprocess,5e5ddf8e-d6f0-4a8d-9ed9-ea2c61e1beda
3,while,1970-01-01 00:00:20,START,subprocess,5e5ddf8e-d6f0-4a8d-9ed9-ea2c61e1beda
4,while,1970-01-01 00:00:40,STOP,subprocess,5e5ddf8e-d6f0-4a8d-9ed9-ea2c61e1beda
5,while,1970-01-01 00:00:40,START,subprocess,5e5ddf8e-d6f0-4a8d-9ed9-ea2c61e1beda
6,while,1970-01-01 00:01:00,STOP,subprocess,5e5ddf8e-d6f0-4a8d-9ed9-ea2c61e1beda
7,while,1970-01-01 00:01:00,START,subprocess,5e5ddf8e-d6f0-4a8d-9ed9-ea2c61e1beda
8,while,1970-01-01 00:01:20,STOP,subprocess,5e5ddf8e-d6f0-4a8d-9ed9-ea2c61e1beda
9,while,1970-01-01 00:01:20,START,subprocess,5e5ddf8e-d6f0-4a8d-9ed9-ea2c61e1beda


The resulting levels of objects in the hopper and the from_site are requested below. 

In [7]:
hopper.container.get_level()

5.0

In [8]:
from_site.container.get_level()

5.0